# COSC 520 - Assignment 1: The Login Checker Problem

## Data Generation

In [2]:
import random
import string

def generate_logins(n, length=6):
    """Generates a list of n random logins.
        - n: number of logins to generate
        - length: length of each login (default is 6)
        Returns a list of unique logins.
    """
    logins = set()
    while len(logins) < n:
        login = ''.join(random.choices(string.ascii_lowercase + string.digits, k=length))
        logins.add(login)
    return list(logins)

generate_logins(10)

['xe7970',
 'a6ocua',
 'mb89dl',
 'casfez',
 'jkryo1',
 'ntww5b',
 'fivest',
 'oozral',
 'z451jk',
 '961x3r']

In [ ]:
# Install required packages
#%!pip install pybloom_live
#%!pip install cuckoofilter
#%pip install pyprobables

Note: you may need to restart the kernel to use updated packages.


## Performance Comparison

A file named "login_all_1B.txt" was created with 1B unique usernames (Drive Link: {https://drive.google.com/file/d/1Yl8NTkzK7QA1cl4inRCTQPCEDcJB22U-/view?usp=sharing}). 

The generation script is added to this repo. 

Later, I will define the functions I will use for loading the data as well as comparing the models.

In [13]:
import time
import bisect
import matplotlib.pyplot as plt
import numpy as np
from pybloom_live import BloomFilter
from probables import CuckooFilter  
import json
import tracemalloc 

In [14]:
def load_logins(filename, num_logins):
    """Reads the first 'num_logins' from the specified file.
        - filename: path to the file containing logins (one per line)
        - num_logins: number of logins to read from the file
        Returns a list of logins.
    """
    logins = []
    print(f"  Reading {num_logins} logins from {filename}...")
    try:
        with open(filename, "r") as f:
            for i, line in enumerate(f):
                if i >= num_logins:
                    break
                logins.append(line.strip())
        if len(logins) < num_logins:
            print(f"  Warning: File contains only {len(logins)} logins, less than requested {num_logins}.")
    except FileNotFoundError:
        print(f"  Error: The file '{filename}' was not found.")
        return []
    return logins

In [15]:
def memory_usage_mb():
    """Return current memory usage in MB using tracemalloc
    """
    current, peak = tracemalloc.get_traced_memory()
    return current / (1024*1024), peak / (1024*1024)


In [16]:
def compare_performance(n_values, filename):
    """Compares performance of different login checkers.
        - n_values: list of n values (number of logins) to test
        - filename: path to the file containing logins (one per line)
                
        Methods compared:
        - Linear Search
        - Binary Search
        - Hashing (Set)
        - Bloom Filter
        - Cuckoo Filter
        
        1. Build Phase: Time and memory to build the data structure.
        2. Lookup Phase: Average time to lookup a login (averaged over multiple lookups).
        3. Memory Usage: Memory used by the data structure after building.
        
        Returns:
        - build_results: dict mapping method names to list of build times
        - lookup_results: dict mapping method names to list of average lookup times
        - memory_results: dict mapping method names to list of memory usage in MB
    """
    
    build_results = {}
    lookup_results = {}
    memory_results = {}

    methods = ["Linear Search", "Binary Search", "Hashing (Set)", "Bloom Filter", "Cuckoo Filter"]
    for m in methods:
        build_results[m] = []
        lookup_results[m] = []
        memory_results[m] = []

    num_lookups = 1000 # Number of lookups to average over
    
    # load logins from the file
    for n in n_values: 
        print(f"\nRunning performance tests for n = {n}...")
        logins = load_logins(filename, n)
        if not logins or len(logins) < n:
            for m in methods:
                build_results[m].append(0)
                lookup_results[m].append(0)
                memory_results[m].append(0)
            continue

        test_login_exists = logins[-1]

        tracemalloc.start()

        # --- Linear Search ---
        print("  Building Linear Search...")
        start_mem = memory_usage_mb()[0]
        start_time = time.time()
        linear_list = list(logins)
        build_results["Linear Search"].append(time.time() - start_time)
        memory_results["Linear Search"].append(memory_usage_mb()[0] - start_mem)

        # --- Binary Search ---
        print("  Building Binary Search...")
        start_mem = memory_usage_mb()[0]
        start_time = time.time()
        sorted_logins = sorted(logins)
        build_results["Binary Search"].append(time.time() - start_time)
        memory_results["Binary Search"].append(memory_usage_mb()[0] - start_mem)

        # --- Hashing (Set) ---
        print("  Building Hashing (Set)...")
        start_mem = memory_usage_mb()[0]
        start_time = time.time()
        login_set = set(logins)
        build_results["Hashing (Set)"].append(time.time() - start_time)
        memory_results["Hashing (Set)"].append(memory_usage_mb()[0] - start_mem)

        # --- Bloom Filter ---
        print("  Building Bloom Filter...")
        start_mem = memory_usage_mb()[0]
        start_time = time.time()
        bloom = BloomFilter(capacity=n, error_rate=0.001)
        for login in logins:
            bloom.add(login)
        build_results["Bloom Filter"].append(time.time() - start_time)
        memory_results["Bloom Filter"].append(memory_usage_mb()[0] - start_mem)

        # --- Cuckoo Filter ---
        print("  Building Cuckoo Filter...")
        start_mem = memory_usage_mb()[0]
        start_time = time.time()
        cuckoo = CuckooFilter(capacity=n, bucket_size=4, finger_size=2)
        inserted_count = 0
        try:    # error handling for cuckoo filter full condition
            cuckoo.add(login)
            inserted_count += 1
        except Exception:
            print(f"Cuckoo filter became full after {inserted_count} insertions.")
            break
        build_results["Cuckoo Filter"].append(time.time() - start_time)
        memory_results["Cuckoo Filter"].append(memory_usage_mb()[0] - start_mem)
        del logins

        # --- Lookup Phase ---
        print("  Timing Linear Search lookups...")
        start_time = time.time()
        for _ in range(num_lookups):
            _ = test_login_exists in linear_list
        lookup_results["Linear Search"].append((time.time() - start_time)/num_lookups)
        del linear_list

        print("  Timing Binary Search lookups...")
        start_time = time.time()
        for _ in range(num_lookups):
            index = bisect.bisect_left(sorted_logins, test_login_exists)
            _ = index < len(sorted_logins) and sorted_logins[index] == test_login_exists
        lookup_results["Binary Search"].append((time.time() - start_time)/num_lookups)
        del sorted_logins

        print("  Timing Hashing (Set) lookups...")
        start_time = time.time()
        for _ in range(num_lookups):
            _ = test_login_exists in login_set
        lookup_results["Hashing (Set)"].append((time.time() - start_time)/num_lookups)
        del login_set

        print("  Timing Bloom Filter lookups...")
        start_time = time.time()
        for _ in range(num_lookups):
            _ = test_login_exists in bloom
        lookup_results["Bloom Filter"].append((time.time() - start_time)/num_lookups)
        del bloom

        print("  Timing Cuckoo Filter lookups...")
        start_time = time.time()
        for _ in range(num_lookups):
            _ = cuckoo.check(test_login_exists)
        lookup_results["Cuckoo Filter"].append((time.time() - start_time)/num_lookups)
        del cuckoo

        tracemalloc.stop()

    return build_results, lookup_results, memory_results

In [17]:
def plot_bar_results(n_values, build_data, lookup_data, memory_data):
    """Plots bar charts for build times, lookup times, and memory usage.
        - n_values: list of n values (number of logins)
        - build_data: dict mapping method names to list of build times
        - lookup_data: dict mapping method names to list of average lookup times
        - memory_data: dict mapping method names to list of memory usage in MB
        Plots and saves the charts as PNG files.
    """
    methods = list(build_data.keys())
    x = np.arange(len(n_values))
    width = 0.15

    # Build Times
    fig1, ax1 = plt.subplots(figsize=(14, 7))
    for i, method in enumerate(methods):
        valid_data = [d for d in build_data[method] if d > 0]
        valid_x = [pos for pos, d in zip(x, build_data[method]) if d > 0]
        if not valid_data: continue
        offset = width * (i - len(methods)/2) + width/2
        rects = ax1.bar(np.array(valid_x)+offset, valid_data, width, label=method)
        ax1.bar_label(rects, padding=3, rotation=90, fmt="%.2e")
    ax1.set_xlabel("Number of Logins (n)")
    ax1.set_ylabel("Build Time (s) - Log Scale")
    ax1.set_title("Data Structure Build Times")
    ax1.set_xticks(x, n_values)
    ax1.set_yscale("log")
    ax1.legend()
    ax1.grid(True, which="both", ls="--", axis="y")
    fig1.tight_layout()
    plt.savefig("build_times_barchart.png")
    plt.show()

    # Lookup Times
    fig2, ax2 = plt.subplots(figsize=(14, 7))
    for i, method in enumerate(methods):
        valid_data = [d for d in lookup_data[method] if d > 0]
        valid_x = [pos for pos, d in zip(x, lookup_data[method]) if d > 0]
        if not valid_data: continue
        offset = width * (i - len(methods)/2) + width/2
        rects = ax2.bar(np.array(valid_x)+offset, valid_data, width, label=method)
        ax2.bar_label(rects, padding=3, rotation=90, fmt="%.2e")
    ax2.set_xlabel("Number of Logins (n)")
    ax2.set_ylabel("Average Lookup Time (s) - Log Scale")
    ax2.set_title("Login Checker Lookup Times")
    ax2.set_xticks(x, n_values)
    ax2.set_yscale("log")
    ax2.legend()
    ax2.grid(True, which="both", ls="--", axis="y")
    fig2.tight_layout()
    plt.savefig("lookup_times_barchart.png")
    plt.show()

    # Memory Usage
    fig3, ax3 = plt.subplots(figsize=(14, 7))
    for i, method in enumerate(methods):
        valid_data = [d for d in memory_data[method] if d > 0]
        valid_x = [pos for pos, d in zip(x, memory_data[method]) if d > 0]
        if not valid_data: continue
        offset = width * (i - len(methods)/2) + width/2
        rects = ax3.bar(np.array(valid_x)+offset, valid_data, width, label=method)
        ax3.bar_label(rects, padding=3, rotation=90, fmt="%.2f MB")
    ax3.set_xlabel("Number of Logins (n)")
    ax3.set_ylabel("Memory Usage (MB)")
    ax3.set_title("Memory Usage per Data Structure")
    ax3.set_xticks(x, n_values)
    ax3.legend()
    ax3.grid(True, ls="--", axis="y")
    fig3.tight_layout()
    plt.savefig("memory_usage_barchart.png")
    plt.show()

In [18]:
def plot_memory_usage_line(n_values, memory_data):
    """
    Plots only the memory usage results as a line plot.
    Args:
        n_values (list): A list of the number of items tested (e.g., [1_000_000, 5_000_000]).
        memory_data (dict): A dictionary containing memory usage results for each method.
    """
    methods = list(memory_data.keys())
    
    # Convert n_values to millions for a cleaner x-axis
    n_values_in_millions = [n / 1_000_000 for n in n_values]

    plt.figure(figsize=(12, 7))
    
    for method in methods:
        # Plot a line for each method with markers on the data points
        plt.plot(n_values_in_millions, memory_data[method], marker='o', linestyle='-', label=method)

    # Add text, title, and labels
    plt.xlabel("Number of Logins (in millions)")
    plt.ylabel("Memory Usage (MB)")
    plt.title("Memory Usage per Data Structure")
    plt.legend()
    plt.grid(True, which="both", ls="--")
    
    plt.tight_layout()
    plt.savefig("memory_usage_lineplot.png")
    plt.show()

In [19]:
def plot_line_results(n_values, build_data, lookup_data, build_filename, lookup_filename, use_log_scale=True, exclude=None):
    """ Generates line plot files for build and lookup times
        - n_values: list of n values (number of logins)
        - build_data: dict mapping method names to list of build times
        - lookup_data: dict mapping method names to list of average lookup times
        - build_filename: filename to save the build times plot
        - lookup_filename: filename to save the lookup times plot
        - use_log_scale: whether to use a logarithmic scale for the y-axis
        - exclude: list of method names to exclude from the plots
        Saves the plots as PNG files.
    """
    if exclude is None:
        exclude = []
    
    # Convert n_values to millions for the x-axis
    n_values_in_millions = [n / 1_000_000 for n in n_values]
    
    methods_to_plot = [m for m in build_data.keys() if m not in exclude]
    scale_type = "Log" if use_log_scale else "Linear"
    excluded_str = f"(Excluding: {', '.join(exclude)})" if exclude else ""
    
    # --- Plot 1: Build Times ---
    plt.figure(figsize=(12, 7))
    for method in methods_to_plot:
        # Use the converted x-axis values
        plt.plot(n_values_in_millions, build_data[method], marker='o', linestyle='-', label=method)
    
    # Update the x-axis label
    plt.xlabel("Number of Logins (in millions)", fontsize=12)
    plt.ylabel("Time (seconds)", fontsize=12)
    plt.title(f"Build Times ({scale_type} Scale) {excluded_str}", fontsize=16)
    plt.legend()
    plt.grid(True, which="both", linestyle='--', linewidth=0.5)
    
    if use_log_scale:
        plt.yscale('log')
        
    plt.tight_layout()
    plt.savefig(build_filename)
    plt.close()
    print(f"Plot saved as {build_filename}")

    # --- Plot 2: Lookup Times ---
    plt.figure(figsize=(12, 7))
    for method in methods_to_plot:
        # Use the converted x-axis values
        plt.plot(n_values_in_millions, lookup_data[method], marker='s', linestyle='-', label=method)

    # Update the x-axis label
    plt.xlabel("Number of Logins (in millions)", fontsize=12)
    plt.ylabel("Time (seconds)", fontsize=12)
    plt.title(f"Lookup Times ({scale_type} Scale) {excluded_str}", fontsize=16)
    plt.legend()
    plt.grid(True, which="both", linestyle='--', linewidth=0.5)

    if use_log_scale:
        plt.yscale('log')
        
    plt.tight_layout()
    plt.savefig(lookup_filename)
    plt.close()
    print(f"Plot saved as {lookup_filename}")

## Running Comparison

In [ ]:
login_file = "logins_all_1B.txt"
n_values_to_test = [1_000_000, 5_000_000, 10_000_000, 50_000_000]
build_times, lookup_times, memory_usage = compare_performance(n_values_to_test, login_file)
plot_bar_results(n_values_to_test, build_times, lookup_times, memory_usage)
plot_memory_usage_line(n_values_to_test, memory_usage)
plot_line_results(
    n_values_to_test, build_times, lookup_times,
    build_filename="build_times_log_scale.png",
    lookup_filename="lookup_times_log_scale.png",
    use_log_scale=True
)

In [ ]:
"""Save results to JSON file"""
all_results = {
    'build_times': build_times,
    'lookup_times': lookup_times,
    'memory_usage': memory_usage
}

with open("performance_results.json", "w") as f:
    json.dump(all_results, f, indent=4)

print("✅ Results saved to performance_results.json")

✅ Results saved to performance_results.json


## Unit Tests

In [24]:
import unittest
import bisect
from pybloom_live import BloomFilter
from probables import CuckooFilter

class TestLoginCheckers(unittest.TestCase):
    """
    A comprehensive suite of unit tests for all login checker methods.
    """

    def setUp(self):
        """
        Set up common data for the tests. This method is run before each test.
        """
        self.logins = ["user1", "test_user", "admin", "guest", "dina579"]
        self.login_exists = "admin"
        self.login_does_not_exist = "unknown_user"

    def test_linear_search(self):
        """
        Tests the linear search method (using a list).
        """
        linear_list = self.logins
        self.assertTrue(self.login_exists in linear_list)
        self.assertFalse(self.login_does_not_exist in linear_list)

    def test_binary_search(self):
        """
        Tests the binary search method on a sorted list.
        """
        sorted_logins = sorted(self.logins)
        
        # Test for an existing login
        index_exists = bisect.bisect_left(sorted_logins, self.login_exists)
        found_exists = (index_exists < len(sorted_logins) and 
                        sorted_logins[index_exists] == self.login_exists)
        self.assertTrue(found_exists)

        # Test for a non-existent login
        index_not_exists = bisect.bisect_left(sorted_logins, self.login_does_not_exist)
        found_not_exists = (index_not_exists < len(sorted_logins) and 
                            sorted_logins[index_not_exists] == self.login_does_not_exist)
        self.assertFalse(found_not_exists)

    def test_hashing_set(self):
        """
        Tests the hashing method (using a set).
        """
        login_set = set(self.logins)
        self.assertTrue(self.login_exists in login_set)
        self.assertFalse(self.login_does_not_exist in login_set)
    
    def test_bloom_filter(self):
        """
        Tests the basic functionality of the Bloom Filter.
        """
        bloom = BloomFilter(capacity=100, error_rate=0.01)
        self.assertFalse('test' in bloom)
        bloom.add('test')
        self.assertTrue('test' in bloom)
        
    def test_cuckoo_filter(self):
        """
        Tests the basic functionality of the Cuckoo Filter, including deletion.
        """
        cuckoo = CuckooFilter(capacity=100)
        self.assertFalse(cuckoo.check('test'))
        cuckoo.add('test')
        self.assertTrue(cuckoo.check('test'))
        cuckoo.remove('test')
        self.assertFalse(cuckoo.check('test'))


unittest.main(argv=['first-arg-is-ignored'], exit=False)


.......
----------------------------------------------------------------------
Ran 7 tests in 0.025s

OK
